In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [8]:
print("No. of GPUS available:",len(tf.config.experimental.list_physical_devices('GPU')))

No. of GPUS available: 1


In [2]:
import pandas as pd
df = pd.read_csv("combined_data.csv")
df.head(5)

,text,label
0,#Depressed mood can be caused by infectious di...,0
1,"With all of this unnessary family drama, I fe...",1
2,"I'm going to keep banging on about this, cos i...",0
3,Ugh. Woke up to the same acne covered face thi...,0
4,#stoner #instahookah #depressed Portable Trans...,0


In [3]:
df.groupby('label').describe()

text                                                               
       count unique                                                top freq
label                                                                      
0      11470   9985    haIZ....frOM laSt wEek i keEP on goiNg out.....    6
1       5709   4373  Mom's depression tied to kids' emotional, inte...   11

In [4]:
df_positive=df[df['label']==1]
df_negative=df[df['label']==0]
df_negative_down = df_negative.sample(df_positive.shape[0])
df_negative_down.shape

df_balanced = pd.concat([df_positive,df_negative_down])
df_balanced

,text,label
1,"With all of this unnessary family drama, I fe...",1
6,"I feel so sad because i can't be happy, and th...",1
7,I went to church with my mom and a huge chunk ...,1
8,No love here.... #lonely #depressed pic.twitt...,1
9,I see so many people moving forward with their...,1
...,...,...
7945,@shaznyc I drink my Corona's sans the fruit. ...,0
2271,did u see american idol man i didn't know w...,0
14059,just played a cash game with my friends at my ...,0
4396,today i nv go sch... sian lohz... hanwen cal...,0


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['text'],df_balanced['label'],stratify=df_balanced['label'])

In [14]:
X_train.head(10)

3754       Life Sux! Another monday liao. What should i...
12019    @MrJoatmon You're very welcome  Where are you ...
2289      Sadness is the only thing getting me through ...
11951    @canzgirl ok, we can sort that out! See ya in ...
547      My #StarWars binge concluded with episode 3 la...
2217       urlLink    Another couple of hotties. I hope...
1137      I willingly admit that I am a sufferer of dep...
7149     @gerardway I can't waight  till the new albam ...
4071       Apr. 22nd Sorry for disappearing for a whole...
12365    @Kaijah Thanks, I appreciate your thoughts and...
Name: text, dtype: object

In [15]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [16]:
def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embedding(["$110 discount, hurry up","Bhavin, are you up for a football game tonight?"])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8321504 , -0.49884483, -0.8328162 , ..., -0.631571  ,
        -0.7261327 ,  0.91106266],
       [-0.8599928 , -0.5069295 , -0.94539136, ..., -0.8701027 ,
        -0.6732962 ,  0.829447  ]], dtype=float32)>

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
#bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
embeddings = bert_encoder(preprocessed_text)
print(embeddings['pooled_output'])
#neural network layers
l = tf.keras.layers.Dropout(0.1, name='dropout')(embeddings['pooled_output'])

l = tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)

#construct final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='keras_layer_1/StatefulPartitionedCall:13', description="created by layer 'keras_layer_1'")


In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'default': (None, 7 109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [20]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
]
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=METRICS)

In [21]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
47/47 [==============================] - 311s 6s/step - loss: 0.6500 - accuracy: 0.6193 - precision: 0.6129 - recall: 0.6480
Epoch 2/10
47/47 [==============================] - 303s 6s/step - loss: 0.5809 - accuracy: 0.6860 - precision: 0.6800 - recall: 0.7027
Epoch 3/10
47/47 [==============================] - 303s 6s/step - loss: 0.5432 - accuracy: 0.7340 - precision: 0.7300 - recall: 0.7427
Epoch 4/10
47/47 [==============================] - 304s 6s/step - loss: 0.5134 - accuracy: 0.7687 - precision: 0.7676 - recall: 0.7707
Epoch 5/10
47/47 [==============================] - 304s 6s/step - loss: 0.4916 - accuracy: 0.7807 - precision: 0.7723 - recall: 0.7960
Epoch 6/10
47/47 [==============================] - 304s 6s/step - loss: 0.4652 - accuracy: 0.7960 - precision: 0.7937 - recall: 0.8000
Epoch 7/10
47/47 [==============================] - 619s 13s/step - loss: 0.4513 - accuracy: 0.8127 - precision: 0.8156 - recall: 0.8080
Epoch 8/10
47/47 [=============================

In [22]:
model.evaluate(X_test,y_test)

16/16 [==============================] - 102s 6s/step - loss: 0.4058 - accuracy: 0.8560 - precision: 0.8248 - recall: 0.9040


[0.4057708978652954,
 0.8560000061988831,
 0.8248175382614136,
 0.9039999842643738]

In [32]:
"""y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5 , 1, 0)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predicted))"""

              precision    recall  f1-score   support

           0       0.89      0.81      0.85       250
           1       0.82      0.90      0.86       250

    accuracy                           0.86       500
   macro avg       0.86      0.86      0.86       500
weighted avg       0.86      0.86      0.86       500

